In [25]:
import numpy as np
import itertools
import requests
import datetime
import time
import json
from datetime import datetime

# Requests 

From GEO EDA sheet:

## DarkSky Sample

1) cheapest seems to be DarkSky. 1000 per day for free 2) $0.0001 per request after that 3) Request: " https://api.darksky.net/forecast/[key]/40.036944,-121.005833,2005-02-02T17:30:00" (my api key removed)

Output is like this: {"latitude":40.036944,"longitude":-121.005833,"timezone":"America/Los_Angeles","currently":{"time":1107394200,"precipIntensity":0,"precipProbability":0,"temperature":50.24,"apparentTemperature":50.41,"dewPoint":13.85,"humidity":0.23,"windSpeed":2.62,"windGust":13.17,"windBearing":24,"uvIndex":0},"hourly":{"data":
...
{"time":1107414000,"precipIntensity":0,"precipProbability":0,"temperature":41.2,"apparentTemperature":41.2,"dewPoint":12.53,"humidity":0.31,"windSpeed":1.56,"windGust":12.79,"windBearing":100,"uvIndex":0}]},"daily":{"data":[{"time":1107331200,"sunriseTime":1107357120,"sunsetTime":1107393960,"moonPhase":0.78,"precipIntensity":0,"precipIntensityMax":0,"precipProbability":0,"temperatureHigh":57.95,"temperatureHighTime":1107384600,"temperatureLow":38.42,"temperatureLowTime":1107439200,"apparentTemperatureHigh":57.45,"apparentTemperatureHighTime":1107384600,"apparentTemperatureLow":38.91,"apparentTemperatureLowTime":1107439200,"dewPoint":14.85,"humidity":0.31,"windSpeed":3.33,"windGust":14.83,"windGustTime":1107390840,"windBearing":76,"uvIndex":0,"uvIndexTime":1107331200,"temperatureMin":35.03,"temperatureMinTime":1107359100,"temperatureMax":57.95,"temperatureMaxTime":1107384600,"apparentTemperatureMin":34.3,"apparentTemperatureMinTime":1107338640,"apparentTemperatureMax":57.45,"apparentTemperatureMaxTime":1107384600}]},"flags":{"sources":["cmc","gfs","hrrr","icon","isd","madis","nam","sref"],"nearest-station":3.312,"units":"us"},"offset":-8}

## Dark Sky locality

In [2]:
# From Geo Sheet - Toy Data Bounding Box
# keep in mind each pixel is roughly 500 m x 500 m
bb_long = [-122, -119.912,-119.912,-122,-122 ]
bb_lat = [36.8, 36.8, 35.06, 35.06,36.8]

Requests, to see how how 'hyperlocal' the darksky data really is.

In [3]:
# Create Pairs
min_long = min(bb_long)
max_long = max(bb_long)
min_lat = min(bb_lat)
max_lat = max(bb_lat)

In [4]:
spaces = 10
longs = np.linspace(min_long,max_long,spaces)
lats = np.linspace(min_lat,max_lat,spaces)
print(longs, lats)
distance = (max_lat-min_lat)*110/spaces
print(f"distance between forecast pairings ~ {distance:.3} km, or {(distance/.5):.3} pixels")

[-122.    -121.768 -121.536 -121.304 -121.072 -120.84  -120.608 -120.376
 -120.144 -119.912] [35.06       35.25333333 35.44666667 35.64       35.83333333 36.02666667
 36.22       36.41333333 36.60666667 36.8       ]
distance between forecast pairings ~ 19.1 km, or 38.3 pixels


In [5]:
# Pairs
pairs = [(lat,long) for lat in lats for long in longs]
print(len(pairs))
print(pairs[25])

100
(35.446666666666665, -120.84)


In [6]:
key = '5ffac5f056d341c6296cba58fa96e9ba'

The Forecast Data API supports HTTP compression. We heartily recommend using it, as it will make responses much smaller over the wire. To enable it, simply add an `Accept-Encoding: gzip` header to your request.

### Try one request

In [7]:
# items to exclude from call
blocks = '[currently,minutely,hourly,alerts]'
# Units for call
units = 'ca'
# Time
time = '2005-02-02T12:00:00' # for noon, but hour doesn't matter as we're grabbing daily data only.
# Dates (relevant to our fires): only use one for testing
dates = ['2016-07-21']

# create time string:
date = dates[0]
time = date+'T12:00:00'
lat = str(pairs[25][0])
long = str(pairs[25][1])

query = ('https://api.darksky.net/forecast/'+key+'/'+ 
        lat+','+long+','+time+'?exclude=' 
        +blocks+'&units='+units)
headers = {'Accept-Encoding':'gzip'}
print(query)   

https://api.darksky.net/forecast/5ffac5f056d341c6296cba58fa96e9ba/35.446666666666665,-120.84,2016-07-21T12:00:00?exclude=[currently,minutely,hourly,alerts]&units=ca


In [8]:
r=requests.get(query,headers=headers)

In [9]:
r.status_code

200

In [10]:
weather = r.json()
weather['daily']['data']

[{'time': 1469084400,
  'summary': 'Foggy in the morning.',
  'icon': 'clear-day',
  'sunriseTime': 1469106300,
  'sunsetTime': 1469157360,
  'moonPhase': 0.58,
  'precipIntensity': 0,
  'precipIntensityMax': 0,
  'precipProbability': 0,
  'temperatureHigh': 32.15,
  'temperatureHighTime': 1469138220,
  'temperatureLow': 13.71,
  'temperatureLowTime': 1469194800,
  'apparentTemperatureHigh': 31.87,
  'apparentTemperatureHighTime': 1469138220,
  'apparentTemperatureLow': 13.98,
  'apparentTemperatureLowTime': 1469194800,
  'dewPoint': 9.95,
  'humidity': 0.56,
  'pressure': 1016.3,
  'windSpeed': 6.1,
  'windGust': 20.82,
  'windGustTime': 1469145600,
  'windBearing': 268,
  'cloudCover': 0.08,
  'uvIndex': 11,
  'uvIndexTime': 1469131740,
  'visibility': 15.083,
  'temperatureMin': 11.36,
  'temperatureMinTime': 1469104980,
  'temperatureMax': 32.15,
  'temperatureMaxTime': 1469138220,
  'apparentTemperatureMin': 11.63,
  'apparentTemperatureMinTime': 1469104980,
  'apparentTemperature

In [11]:
# Parsed important elements:
dailydata = weather['daily']['data'][0]
rain = dailydata['precipIntensityMax']
hitemp = dailydata['temperatureHigh']
lotemp = dailydata['temperatureLow']
humidity = dailydata['humidity']
windspd = dailydata['windSpeed']
winddir = dailydata['windBearing']
clouds = dailydata['cloudCover']

In [12]:
point = [date,lat,long,rain,hitemp,lotemp,humidity,windspd,winddir,clouds]

In [13]:
point

['2016-07-21',
 '35.446666666666665',
 '-120.84',
 0,
 32.15,
 13.71,
 0.56,
 6.1,
 268,
 0.08]

In [14]:
print(type(point))

<class 'list'>


In [15]:
pairs[0][0]

35.06

## Multiple Requests to determine degree of hyperlocality required

In [16]:
%%time
# Create requests
# for date in dates: (only doing 1 date for now)
data_out = []
for pair in pairs:
    lat = str(pair[0])
    long = str(pair[1])
    query = ('https://api.darksky.net/forecast/'+key+'/'+ 
        lat+','+long+','+time+'?exclude=' 
        +blocks+'&units='+units)
    r=requests.get(query,headers=headers)
    weather=r.json()
    data_out.append(weather)
    

In [24]:
print(data_out[50])
x = data_out[0].get('daily')
print(x)

{'latitude': 36.026666666666664, 'longitude': -122, 'timezone': 'America/Los_Angeles', 'daily': {'data': [{'time': 1469084400, 'sunriseTime': 1469106540, 'sunsetTime': 1469157720, 'moonPhase': 0.58, 'precipIntensity': 0, 'precipIntensityMax': 0, 'precipProbability': 0, 'temperatureHigh': 21.38, 'temperatureHighTime': 1469137140, 'temperatureLow': 12.03, 'temperatureLowTime': 1469190480, 'apparentTemperatureHigh': 21.1, 'apparentTemperatureHighTime': 1469137140, 'apparentTemperatureLow': 12.3, 'apparentTemperatureLowTime': 1469190480, 'dewPoint': 9.97, 'humidity': 0.68, 'pressure': 1016.8, 'windSpeed': 14.98, 'windGust': 30.73, 'windGustTime': 1469148300, 'windBearing': 321, 'uvIndex': 0, 'uvIndexTime': 1469084400, 'temperatureMin': 12.6, 'temperatureMinTime': 1469100840, 'temperatureMax': 21.38, 'temperatureMaxTime': 1469137140, 'apparentTemperatureMin': 12.87, 'apparentTemperatureMinTime': 1469100840, 'apparentTemperatureMax': 21.1, 'apparentTemperatureMaxTime': 1469137140}]}, 'flags'

In [27]:
## SOME OF THE POINTS ARE IN THE OCEAN , so THERE IS NO DATA.
data_convert=[]
for point in data_out:
    lat = point['latitude']
    long = ['longitude']
    daily = point.get('daily')
    if daily:
        data = daily['data'][0]
        time = datetime.fromtimestamp(data['time']).strftime('%Y-%m-%d')
        rain = data['precipIntensityMax']
        hitemp = data['temperatureHigh']
        lotemp = data['temperatureLow']
        humidity = data['humidity']
        windspd = data['windSpeed']
        winddir = data['windBearing']
        clouds = data['cloudCover']
        point = [date,lat,long,rain,hitemp,lotemp,humidity,windspd,winddir,clouds] 
        data_convert.append(point)
    else:
        point = -9999
    
#     data_out.append(point)
    

KeyError: 'precipIntensityMax'

In [28]:
data_convert

[]